In [2]:
!pip install azureml-core azureml


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\deanv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from azureml.core import Workspace, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Dataset


In [4]:
subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "Y2D-Staff-2025"

In [5]:
auth = InteractiveLoginAuthentication()
workspace = Workspace(subscription_id=subscription_id,
                   resource_group=resource_group,
                   workspace_name=workspace_name,
                   auth=auth,
                   )

The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Performing interactive authentication. Please follow the instructions on the terminal.


Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/8ad04a9f-1a10-4808-9480-b8a2f8a0da14', 'tenant_id': '8ad04a9f-1a10-4808-9480-b8a2f8a0da14', 'tenant_category': 'Home', 'country': None, 'country_code': 'NL', 'display_name': 'SURF', 'domains': ['surf.nl', 'surf.onmicrosoft.com', 'surf.mail.onmicrosoft.com', 'surfsara.com', 'lcrdm.nl', 'versnellingsplan.nl', 'integraalveilig-ho.nl', 'plekkie.nl', 'netherlight.nl', 'netherlight.net', 'surfconext.nl', 'surfaudit.nl', 'surfdashboard.nl', 'surffilesender.nl', 'surfdrive.nl', 'base27.surf.nl', 'surfnet.nl', 'surfsharekit.nl', 'sbcb10169.pstnaccess.cellip.com', 'sbca10169.pstnaccess.cellip.com', 'surfcumulus.nl', 'auteursrechten.nl', 'hbokennisbank.nl', 'openconext.org', 'test.ia.surf.nl', 'ia.surf.nl', 'digitaliseringsimpulsonderwijs.nl', 'sram.surf.nl', 'publinova.nl', 'surfdiensten.nl', 'surfspot.nl', 'surfmarket.nl', 'dcc-po.nl', 'npuls.nl', 'eduid.nl', 'edusources.nl', 'tdcc.nl', 'dnssecplatform.nl', 'honeyspider.net',

Interactive authentication successfully completed.


In [6]:
# list all datastores registered in the current workspace
datastores = workspace.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

workspaceblobstore AzureBlob
workspaceartifactstore AzureBlob
workspaceworkingdirectory AzureFile


<generator object walk_directory at 0x0000023541B71E40>


In [12]:
# Create a datastore object from the existing datastore named "workspaceblobstore".
datastore = Datastore(workspace, name='workspaceblobstore')

# Upload the data to the path target_path in datastore
datastore.upload(src_dir='../data/audio_chunks_sample', target_path='audio_sample', overwrite=True, show_progress=True)

Uploading an estimated of 5 files
Uploading ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ Episode 2_chunk_4_300_seconds.wav
Uploaded ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ Episode 2_chunk_4_300_seconds.wav, 1 files out of an estimated total of 5
Uploading ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ Episode 2_chunk_0_300_seconds.wav
Uploaded ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ Episode 2_chunk_0_300_seconds.wav, 2 files out of an estimated total of 5
Uploading ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ Episode 2_chunk_1_300_seconds.wav
Uploaded ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ Episode 2_chunk_1_300_seconds.wav, 3 files out of an estimated total of 5
Uploading ../data/audio_chunks_sample\A Show of Scrutiny ｜ Critical Role： THE MIGHTY NEIN ｜ E

$AZUREML_DATAREFERENCE_ab0a71a27271487e8d0221453b2b5da4

# This is nice with SDKV1 but doesnt play nice with NEMO. The SDK2 way is way too complicated

In [16]:
# Create a FileDataset from a path to a directory.
# The directory contains a folder per class, each of which contains image files.
sample_set = Dataset.File.from_files(path=(datastore, 'transcripts'))
paths = sample_set.take_sample(0.001).take(30).download()
print(paths)

2025-05-25 13:51:19.189271 | ActivityCompleted: Activity=from_files, HowEnded=Failure, Duration=0.0 [ms], Info = {'activity_id': 'e1620620-bfd9-456b-b432-80bdc0dd4daa', 'activity_name': 'from_files', 'activity_type': 'PublicApi', 'app_name': 'FileDataset', 'source': 'azureml.dataset', 'version': '1.60.0', 'dataprepVersion': '', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': ''}, Exception=ImportError; Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade


ImportError: Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade

In [20]:
# Create a FileDataset from a path to a directory for the training data.
train_set = Dataset.File.from_files(path=(datastore, 'transcripts/train'))
# Split the dataset into train and validation sets
# train_set, val_set = train_set.random_split(0.8, seed=123)
val_set = Dataset.File.from_files(path=(datastore, 'transcripts/train'))
# Create a FileDataset from a path to a directory for the test data.
test_set = Dataset.File.from_files(path=(datastore, 'transcripts/test'))

#register the datasets
train_reg = train_set.register(workspace=workspace, name='transcripts_train', description='training data', create_new_version=True)
val_reg = val_set.register(workspace=workspace, name='transcripts_val', description='validation data', create_new_version=True)
test_reg = test_set.register(workspace=workspace, name='transcriptss_test', description='test data', create_new_version=True)

2025-05-25 14:02:55.082611 | ActivityCompleted: Activity=from_files, HowEnded=Failure, Duration=0.0 [ms], Info = {'activity_id': '444fff19-5219-4594-a6d7-a8cf31e2a0bc', 'activity_name': 'from_files', 'activity_type': 'PublicApi', 'app_name': 'FileDataset', 'source': 'azureml.dataset', 'version': '1.60.0', 'dataprepVersion': '', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': ''}, Exception=ImportError; Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade


ImportError: Missing required package "azureml-dataset-runtime", which can be installed by running: "c:\Users\deanv\Dropbox\0_Buas\2024-2025\Y2D\Demo Project\mlops-demo-project\backend-microservice\.venv\Scripts\python.exe" -m pip install azureml-dataset-runtime --upgrade

In [13]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import InteractiveBrowserCredential
import random

In [14]:
# Initialize client
ml_client = MLClient(
    credential=InteractiveBrowserCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)


In [15]:
# Get datastore reference
datastore = ml_client.datastores.get("workspaceblobstore")  # or your datastore name

In [ ]:
# Create data URIs (equivalent to Dataset.File.from_files)
train_uri = f"azureml://datastores/{datastore.name}/paths/transcript/train"
val_uri = f"azureml://datastores/{datastore.name}/paths/transcript/validaion"
test_uri = f"azureml://datastores/{datastore.name}/paths/transcript/test"

In [40]:
train_data = Data(
    name="transcripts_train",
    description="training data",
    path=train_uri,
    type=AssetTypes.URI_FOLDER
)

val_data = Data(
    name="transcripts_val",
    description="validation data",
    path=val_uri,
    type=AssetTypes.URI_FOLDER
)

test_data = Data(
    name="transcripts_test",
    description="test data", 
    path=test_uri,
    type=AssetTypes.URI_FOLDER
)

# Register the datasets
train_reg = ml_client.data.create_or_update(train_data)
val_reg = ml_client.data.create_or_update(val_data)
test_reg = ml_client.data.create_or_update(test_data)

print(f"Registered datasets:")
print(f"Train: {train_reg.name} v{train_reg.version}")
print(f"Val: {val_reg.name} v{val_reg.version}")
print(f"Test: {test_reg.name} v{test_reg.version}")

Registered datasets:
Train: transcripts_train v1
Val: transcripts_val v1
Test: transcripts_test v1


In [18]:
train_file = "../data/training_data/train.csv"
val_file = "../data/training_data/val.csv"
test_file = "../data/training_data/test.csv"

In [19]:
# Create Data assets
train_data = Data(
    name="transcripts_train",
    description="Critical Role transcript training data for summarization",
    path=train_file,
    type=AssetTypes.URI_FILE
)

val_data = Data(
    name="transcripts_val", 
    description="Critical Role transcript validation data for summarization",
    path=val_file,
    type=AssetTypes.URI_FILE
)

test_data = Data(
    name="transcripts_test",
    description="Critical Role transcript test data for summarization",
    path=test_file,
    type=AssetTypes.URI_FILE
)

print("Registering datasets with Azure ML...")

# Register the datasets
train_reg = ml_client.data.create_or_update(train_data)
print(f"✓ Registered training dataset: {train_reg.name} v{train_reg.version}")

val_reg = ml_client.data.create_or_update(val_data)
print(f"✓ Registered validation dataset: {val_reg.name} v{val_reg.version}")

test_reg = ml_client.data.create_or_update(test_data)
print(f"✓ Registered test dataset: {test_reg.name} v{test_reg.version}")

print(f"\n🎉 Successfully registered all datasets!")
print(f"Train: {train_reg.name} v{train_reg.version}")
print(f"Val: {val_reg.name} v{val_reg.version}")
print(f"Test: {test_reg.name} v{test_reg.version}")

Registering datasets with Azure ML...


Uploading train.csv (< 1 MB): 100%|##########| 162k/162k [00:00<00:00, 1.27MB/s]




✓ Registered training dataset: transcripts_train v1


Uploading val.csv (< 1 MB): 100%|##########| 43.8k/43.8k [00:00<00:00, 787kB/s]




✓ Registered validation dataset: transcripts_val v1


Uploading test.csv (< 1 MB): 100%|##########| 49.0k/49.0k [00:00<00:00, 635kB/s]




✓ Registered test dataset: transcripts_test v1

🎉 Successfully registered all datasets!
Train: transcripts_train v1
Val: transcripts_val v1
Test: transcripts_test v1


In [20]:
# list all datasets registered in the current workspace
datasets = workspace.datasets
for name, dataset in datasets.items():
    print(name)

In [21]:
# List all data assets
print("All registered data assets:")
print("-" * 50)

for data_asset in ml_client.data.list():
    print(f"📊 {data_asset.name}")
    print(f"   Version: {data_asset.version}")
    print(f"   Type: {data_asset.type}")
    print(f"   Description: {data_asset.description}")
    print(f"   Path: {data_asset.path}")
    print(f"   Created: {data_asset.creation_context.created_at}")
    print()

All registered data assets:
--------------------------------------------------
📊 transcripts_train
   Version: None
   Type: uri_file
   Description: None
   Path: None
   Created: 2025-06-01 14:49:32.272968+00:00

📊 transcripts_val
   Version: None
   Type: uri_file
   Description: None
   Path: None
   Created: 2025-06-01 14:49:34.190696+00:00

📊 transcripts_test
   Version: None
   Type: uri_file
   Description: None
   Path: None
   Created: 2025-06-01 14:49:36.175757+00:00



In [24]:
import pandas as pd

data_asset = ml_client.data.get("transcripts_train", version="1")

df = pd.read_csv(data_asset.path)
df

ValueError: Protocol not known: azureml

In [25]:
import pandas as pd

# Get the data asset
data_asset = ml_client.data.get("transcripts_train", version="1")

# WRONG (what Azure ML Studio suggests but doesn't work):
# df = pd.read_csv(data_asset.path)  # This fails with "Protocol not known: azureml"

# CORRECT - Download first, then read:
import tempfile
import os

# Download the data asset to a temporary location
temp_dir = tempfile.mkdtemp()
downloaded_path = ml_client.data.download(
    name="transcripts_train",
    version="1",
    download_path=temp_dir
)

# Find the CSV file in the downloaded folder
csv_file = None
for root, dirs, files in os.walk(downloaded_path):
    for file in files:
        if file.endswith('.csv'):
            csv_file = os.path.join(root, file)
            break

# Now read the CSV
df = pd.read_csv(csv_file)
df

AttributeError: 'DataOperations' object has no attribute 'download'